In [1]:
from pre_process import pre_process_data
from feature_engineeing import build_features_for_analytics, build_features_for_rag

c:\Users\91790\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import pandas as pd 
data = pd.read_csv('hotel_bookings.csv')

data = pre_process_data(data)

In [4]:
data, gm = build_features_for_rag(data, verbose=True)

Processing Revenue Trends...
Processing Booking and Cancellations Trends...
Processing Occupancy and Demand Trends...
Processing Customer and Demographic Features...
Processing Time Based Trends...


Now, We have our dataframe and Global metrics ready

Thus, We can now build text data over these!

**Processing DataFrame**

In [3]:
data.columns

Index(['hotel', 'is_canceled', 'lead_time', 'year', 'month',
       'arrival_date_week_number', 'day', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'arrival_date',
       'is_holiday_season', 'is_weekend_arrival'],
      dtype='object')

In [5]:
def row_to_text(row):
    return (
        f"A guest booked a {row['hotel']} with {row['adults']} adults, {row['children']} children, and {row['babies']} babies. "
        f"The booking was made through {row['market_segment']} via {row['distribution_channel']} from {row['country']}. "
        f"The lead time was {row['lead_time']} days (categorized as {row['lead_time_bins']}), and the arrival date was {row['arrival_date']} "
        f"(Year: {row['year']}, Month: {row['month']}, Day: {row['day']}, Week {row['arrival_date_week_number']}). "
        f"The guest stayed {row['stays_in_week_nights']} nights on weekdays and {row['stays_in_weekend_nights']} nights on weekends. "
        f"The reserved room type was {row['reserved_room_type']}, and the assigned room type was {row['assigned_room_type']}. "
        f"The booking had {row['booking_changes']} changes, with {row['previous_cancellations']} previous cancellations and "
        f"{row['previous_bookings_not_canceled']} successful bookings before. "
        f"The deposit type was {row['deposit_type']}, and the booking was handled by agent {row['agent']}. "
        f"The total revenue generated was {row['total_revenue']} with an ADR (Average Daily Rate) of {row['adr']}. "
        f"The reservation status was {row['reservation_status']} on {row['reservation_status_date']}. "
        f"The guest had {row['total_of_special_requests']} special requests and required {row['required_car_parking_spaces']} parking spaces. "
        f"The customer type was {row['customer_type']}, and they spent an average of {row['average_stay_duration']} days per stay. "
        f"The booking was {'canceled' if row['is_canceled'] else 'not canceled'}. "
        f"They were {'a repeated guest' if row['is_repeated_guest'] else 'a first-time guest'}. "
        f"The arrival was {'on a weekend' if row['is_weekend_arrival'] else 'on a weekday'}, and it was {'during' if row['is_holiday_season'] else 'not during'} a holiday season. "
        f"The guest waited {row['waiting_list_days']} days on the waiting list (Total: {row['days_in_waiting_list']} days)."
    )

In [6]:
# Apply text conversion
data["text_data"] = data.apply(row_to_text, axis=1)

In [7]:
data['text_data'][0]

'A guest booked a Resort Hotel with 2 adults, 0.0 children, and 0 babies. The booking was made through Direct via Direct from PRT. The lead time was 342 days (categorized as Long), and the arrival date was 2015-07-01 00:00:00 (Year: 2015, Month: 7, Day: 1, Week 27). The guest stayed 0 nights on weekdays and 0 nights on weekends. The reserved room type was C, and the assigned room type was C. The booking had 3 changes, with 0 previous cancellations and 0 successful bookings before. The deposit type was No Deposit, and the booking was handled by agent 14.0. The total revenue generated was 0.0 with an ADR (Average Daily Rate) of 0.0. The reservation status was Check-Out on 01-07-15. The guest had 0 special requests and required 0 parking spaces. The customer type was Transient, and they spent an average of 0 days per stay. The booking was not canceled. They were a first-time guest. The arrival was on a weekday, and it was during a holiday season. The guest waited 0 days on the waiting lis

In [9]:
text_df = data['text_data']

In [11]:
text_df.to_csv('saved_dfs/text_data.csv', index=False)

**Processing Global Metrics**

In [15]:
for i in gm.keys():
    print(i)

average_revenue_per_booking
revenue_per_month
revenue_per_market_segment
revenue_per_meal_plan
cancellations_by_hotel
cancellations_by_country
cancellations_by_customer_type
cancellations_by_season
overall_cancellation_rate
average_stay_duration
occupancy_rate_per_hotel
demand_per_market_segment
percentage_families
percentage_repeated_guests
special_requests_avg
booking_trend_over_time
busiest_weeks
holiday_season_effect
waiting_list_trend
average_waiting_list_days


In [16]:
average_revenue_per_booking = gm.get('average_revenue_per_booking')

---

In [ ]:
revenue_per_month = gm.get('revenue_per_month')

In [21]:
import calendar
def func1(row):
    month_name = calendar.month_name[row['month']]
    return f"In {row['year']}, the total revenue for the month of {month_name} was {row['total_revenue']:.2f} USD. This reflects the booking revenue generated by the hotel during this period."

In [26]:
# Convert all float to int 
revenue_per_month = revenue_per_month.astype(int)

In [27]:
revenue_per_month['text_data'] = revenue_per_month.apply(func1, axis=1)

In [29]:
# Drop previous cols 
revenue_per_month = revenue_per_month.drop(columns=['year', 'month', 'total_revenue'])

----

In [32]:
revenue_per_market_segment = gm.get('revenue_per_market_segment')

In [33]:
revenue_per_market_segment

,market_segment,total_revenue
0,Aviation,24073.68
1,Complementary,3785.29
2,Corporate,476668.85
3,Direct,3250387.29
4,Groups,2848790.96
5,Offline TA/TO,3987887.18
6,Online TA,14941809.06
7,Undefined,78.00


In [34]:
def func2(row):
    return f"The {row['market_segment']} segment generated a total revenue of {row['total_revenue']:.2f} USD from hotel bookings."

In [35]:
revenue_per_market_segment['text_data'] = revenue_per_market_segment.apply(func2, axis=1)
# drop previous cols
revenue_per_market_segment = revenue_per_market_segment.drop(columns=['market_segment', 'total_revenue'])

---

In [44]:
revenue_per_meal_plan = gm.get('revenue_per_meal_plan')

In [45]:
revenue_per_meal_plan

,meal,total_revenue
0,BB,19256210.43
1,FB,199229.25
2,HB,3876016.85
3,SC,1979224.34
4,Undefined,222799.44


In [48]:
def func3(row):
    return f"The {row['meal']} meal plan generated a total revenue of {row['total_revenue']:.2f} USD from hotel bookings."

In [49]:
revenue_per_meal_plan['text_data'] = revenue_per_meal_plan.apply(func3, axis=1)
# drop previous cols
revenue_per_meal_plan = revenue_per_meal_plan.drop(columns=['meal', 'total_revenue'])

---

In [51]:
cancellations_by_hotel = gm.get('cancellations_by_hotel')

In [52]:
cancellations_by_hotel

,hotel,is_canceled
0,City Hotel,33102
1,Resort Hotel,11122


In [54]:
def func4(row):
    return f"The {row['hotel']} hotel had a total of {row['is_canceled']} cancellations."

In [55]:
cancellations_by_hotel['text_data'] = cancellations_by_hotel.apply(func4, axis=1)
# drop previous cols
cancellations_by_hotel = cancellations_by_hotel.drop(columns=['hotel', 'is_canceled'])

---

In [57]:
cancellations_by_country = gm.get('cancellations_by_country')

In [58]:
cancellations_by_country

,country,is_canceled
0,ABW,0
1,AGO,205
2,AIA,0
3,ALB,2
4,AND,5
...,...,...
172,VGB,1
173,VNM,2
174,ZAF,31
175,ZMB,1


In [59]:
def func5(row):
    return f"The country {row['country']} had a total of {row['is_canceled']} cancellations."

In [60]:
cancellations_by_country['text_data'] = cancellations_by_country.apply(func5, axis=1)
# drop previous cols
cancellations_by_country = cancellations_by_country.drop(columns=['country', 'is_canceled'])

---

In [62]:
cancellations_by_customer_type = gm.get('cancellations_by_customer_type')

In [63]:
cancellations_by_customer_type

,customer_type,is_canceled
0,Contract,1262
1,Group,59
2,Transient,36514
3,Transient-Party,6389


In [64]:
def func6(row):
    return f"The {row['customer_type']} customer type had a total of {row['is_canceled']} cancellations."

In [65]:
cancellations_by_customer_type['text_data'] = cancellations_by_customer_type.apply(func6, axis=1)
# drop previous cols
cancellations_by_customer_type = cancellations_by_customer_type.drop(columns=['customer_type', 'is_canceled'])

---

In [67]:
cancellations_by_season = gm.get('cancellations_by_season')

In [69]:
cancellations_by_season

,is_holiday_season,is_canceled
0,False,30065
1,True,14159


In [70]:
def func7(row):
    return f"The {row['is_holiday_season']} season had a total of {row['is_canceled']} cancellations."

In [71]:
cancellations_by_season['text_data'] = cancellations_by_season.apply(func7, axis=1)
# drop previous cols
cancellations_by_season = cancellations_by_season.drop(columns=['is_holiday_season', 'is_canceled'])

---

In [72]:
overall_cancellation_rate = gm.get('overall_cancellation_rate')
print(f"The overall cancellation rate for hotel bookings was {overall_cancellation_rate:.2f}.")

The overall cancellation rate for hotel bookings was 37.04.


In [73]:
average_stay_duration = gm.get('average_stay_duration')
print(f"The average stay duration for hotel bookings was {average_stay_duration:.2f} days.")

The average stay duration for hotel bookings was 3.43 days.


--- 

In [74]:
occupancy_rate_per_hotel = gm.get('occupancy_rate_per_hotel')

In [75]:
occupancy_rate_per_hotel

,hotel,occupancy_rate
0,City Hotel,0.582730
1,Resort Hotel,0.722366


In [76]:
def func8(row):
    return f"The {row['hotel']} hotel had an occupancy rate of {row['occupancy_rate']:.2f}%."

In [77]:
occupancy_rate_per_hotel['text_data'] = occupancy_rate_per_hotel.apply(func8, axis=1)
# drop previous cols
occupancy_rate_per_hotel = occupancy_rate_per_hotel.drop(columns=['hotel', 'occupancy_rate'])

---

In [78]:
demand_per_market_segment = gm.get('demand_per_market_segment')

In [79]:
demand_per_market_segment

,market_segment,booking_count
0,Online TA,56477
1,Offline TA/TO,24219
2,Groups,19811
3,Direct,12606
4,Corporate,5295
5,Complementary,743
6,Aviation,237
7,Undefined,2


In [80]:
def func9(row):
    return f"The {row['market_segment']} market segment had a total of {row['booking_count']} bookings."

In [81]:
demand_per_market_segment['text_data'] = demand_per_market_segment.apply(func9, axis=1)
# drop previous cols
demand_per_market_segment = demand_per_market_segment.drop(columns=['market_segment', 'booking_count'])

---

In [85]:
percentage_families = gm.get('percentage_families')
print(f"The percentage of family bookings was {percentage_families:.2f}%.")

The percentage of family bookings was 12.21%.


In [86]:
percentage_repeated_guests = gm.get('percentage_repeated_guests')
print(f"The percentage of repeated guests was {percentage_repeated_guests:.2f}%.")

The percentage of repeated guests was 3.19%.


In [87]:
special_requests_avg = gm.get('special_requests_avg')
print(f"The average number of special requests per booking was {special_requests_avg:.2f}.")

The average number of special requests per booking was 0.57.


In [90]:
booking_trend_over_time = gm.get('booking_trend_over_time')

In [92]:
booking_trend_over_time.head(3)

,year,month,total_bookings
0,2015,7,2776
1,2015,8,3889
2,2015,9,5114


In [93]:
def func10(row):
    return f"In {row['year']}, the total number of bookings for {row['month']} was {row['total_bookings']}."

In [94]:
booking_trend_over_time['text_data'] = booking_trend_over_time.apply(func10, axis=1)
# drop previous cols
booking_trend_over_time = booking_trend_over_time.drop(columns=['year', 'month', 'total_bookings'])

---

In [95]:
busiest_weeks = gm.get('busiest_weeks')

In [97]:
busiest_weeks.head(3)

,week_number,total_bookings
0,1,1047
1,2,1218
2,3,1319


In [98]:
def func11(row):
    return f"The week {row['week_number']} had a total of {row['total_bookings']} bookings."

In [99]:
# Get top 10 busiest weeks
# arrange in descending order of total_bookings 
busiest_weeks = busiest_weeks.sort_values(by='total_bookings', ascending=False)

---

In [102]:
holiday_season_effect = gm.get('holiday_season_effect')

In [103]:
holiday_season_effect

,is_holiday_season,total_bookings
0,False,80143
1,True,39247


In [104]:
def func12(row):
    return f"The {row['is_holiday_season']} season had a total of {row['total_bookings']} bookings."

In [105]:
holiday_season_effect['text_data'] = holiday_season_effect.apply(func12, axis=1) 
# drop previous cols
holiday_season_effect = holiday_season_effect.drop(columns=['is_holiday_season', 'total_bookings'])

--- 

In [107]:
waiting_list_trend = gm.get('waiting_list_trend')

In [109]:
waiting_list_trend.head(3)

,year,month,average_waiting_list_days
0,2015,7,0.000000
1,2015,8,0.000000
2,2015,9,0.896558


In [110]:
def func13(row):
    return f"In {row['year']}, the total number of days on the waiting list for {row['month']} was {row['average_waiting_list_days']}."

In [111]:
waiting_list_trend['text_data'] = waiting_list_trend.apply(func13, axis=1)
# drop previous cols
waiting_list_trend = waiting_list_trend.drop(columns=['year', 'month', 'average_waiting_list_days'])

------

**Merge each global metrics data into a single DataFrame**

In [117]:
# Merge all dataframes into a single df 
global_metrics_df = pd.concat([
    revenue_per_month, revenue_per_market_segment, revenue_per_meal_plan, cancellations_by_hotel, cancellations_by_country,
    cancellations_by_customer_type, cancellations_by_season, occupancy_rate_per_hotel, demand_per_market_segment,
    booking_trend_over_time, busiest_weeks, holiday_season_effect, waiting_list_trend
])

In [ ]:
# Save all dataframes to csv
revenue_per_month.to_csv('saved_dfs/revenue_per_month.csv', index=False)
revenue_per_market_segment.to_csv('saved_dfs/revenue_per_market_segment.csv', index=False)
revenue_per_meal_plan.to_csv('saved_dfs/revenue_per_meal_plan.csv', index=False)
cancellations_by_hotel.to_csv('saved_dfs/cancellations_by_hotel.csv', index=False)
cancellations_by_country.to_csv('saved_dfs/cancellations_by_country.csv', index=False)
cancellations_by_customer_type.to_csv('saved_dfs/cancellations_by_customer_type.csv', index=False)
cancellations_by_season.to_csv('saved_dfs/cancellations_by_season.csv', index=False)
occupancy_rate_per_hotel.to_csv('saved_dfs/occupancy_rate_per_hotel.csv', index=False)
demand_per_market_segment.to_csv('saved_dfs/demand_per_market_segment.csv', index=False)
booking_trend_over_time.to_csv('saved_dfs/booking_trend_over_time.csv', index=False)
busiest_weeks.to_csv('saved_dfs/busiest_weeks.csv', index=False)
holiday_season_effect.to_csv('saved_dfs/holiday_season_effect.csv', index=False)
waiting_list_trend.to_csv('saved_dfs/waiting_list_trend.csv', index=False)

In [119]:
# Remove any extra columns other than 'text_data' 
global_metrics_df = global_metrics_df[['text_data']]

In [123]:
global_metrics_df.sample(5)

,text_data
167,The country UMI had a total of 1 cancellations.
110,The country MLT had a total of 5 cancellations.
1,The FB meal plan generated a total revenue of ...
3,The country ALB had a total of 2 cancellations.
20,"In 2017.0, the total number of days on the wai..."


In [ ]:
# Make dataFrame with proper text details based on Single metrics
single_metrics = pd.DataFrame({
    'text_data': [
        f"The average revenue per booking was {average_revenue_per_booking:.2f} USD.",
        f"The overall cancellation rate for hotel bookings was {overall_cancellation_rate:.2f}.",
        f"The average stay duration for hotel bookings was {average_stay_duration:.2f} days.",
        f"The percentage of family bookings was {percentage_families:.2f}%.",
        f"The percentage of repeated guests was {percentage_repeated_guests:.2f}%.",
        f"The average number of special requests per booking was {special_requests_avg:.2f}."
    ]
})

In [128]:
# merge to global df at the same column 'text_data'
global_metrics_df = pd.concat([global_metrics_df, single_metrics])

In [129]:
global_metrics_df.tail(4)

,text_data
2,The average stay duration for hotel bookings w...
3,The percentage of family bookings was 12.21%.
4,The percentage of repeated guests was 3.19%.
5,The average number of special requests per boo...


In [131]:
# save 
global_metrics_df.to_csv('saved_dfs/global_metrics_df.csv', index=False)